# Voxelization-->Meshlabserver--> Skeleton

In [1]:
#import modules and set up datajoint

In [2]:
import datajoint as dj
import numpy as np
import calcification_Module as cm
import os
import pathlib
import subprocess
import time

#for the voxelization:
from meshparty import trimesh_io

#for supressing the output
import os, contextlib

In [3]:
#connect to the database and pull down Cleansed Mesh ones and generate the resurfacing of these
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [4]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [5]:
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/temp")) == False:
    os.mkdir("temp")

In [6]:
def skeletonize_Neuron_Meshlabserver(key):
    start_time_inner = time.time()
    vertices_ar,triangles_ar = (ta3p100.Decimation35 & key).fetch("vertices","triangles")

    vertices = vertices_ar[0]
    triangles = triangles_ar[0]
    print(f"Import time = {time.time() - start_time_inner}")
    
    #voxelize the mesh
    start_time_inner = time.time()
    print("starting voxelization")
    mesh = trimesh_io.Mesh(vertices=vertices, faces=triangles)
    voxels = mesh.voxelized(500)
    voxel_mesh = voxels.as_boxes()
    print("DONE voxelization")
    print(f"Voxelization time = {time.time() - start_time_inner}")
    
    #check that there is a temporary file to store it
    
    start_time_inner = time.time()
    
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    #try the inline printing method:
    with open(os.devnull, 'w') as devnull:
        with contextlib.redirect_stdout(devnull):
            voxel_mesh.export(str(path_and_filename) + ".off")
    print(f"Writing Off file VOXEL time = {time.time() - start_time_inner}")
    
    
    #save off the voxelized mesh as .off 
    #so it can be passed to meshlab server
    
    """  SLOWER THAN THE EXPORT FUNCTION OF THE VOXEL_MESH BY 2 SECONDS
    start_time_inner = time.time()
    print("starting writing neuron to off file")
    paths = write_Whole_Neuron_Off_file(str(key['segment_id'])+"_vox",voxel_mesh.vertices,voxel_mesh.faces)
    print("DONE writing neuron to off file")
    print(f"Writing Off file time = {time.time() - start_time_inner}")
    """
    
    start_time_inner = time.time()
    input_mesh = path_and_filename + ".off"
    midoutput_mesh = path_and_filename + "_mid.off"
    output_mesh = path_and_filename+"_mls.off"
    
    meshlab_script = str(file_loc) + "/" + "remeshing_script_vox.mlx"
    meshlab_script_rem_dupl = str(file_loc) + "/" + "remeshing_script_vox_rem_dupl.mlx"
    #send to meshlabserver
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result = run_meshlab_script(meshlab_script,
                      input_mesh,
                      midoutput_mesh)
    print("subprocess_result= "+ str(subprocess_result))
    
    #do another call to remove the final duplicate vertices
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result = run_meshlab_script(meshlab_script_rem_dupl,
                      midoutput_mesh,
                      output_mesh)
    print("subprocess_result= "+ str(subprocess_result))
    
    print("DONE meshlabserver Poisson surface reconstruction")
    print(f"Meshlabserver time = {time.time() - start_time_inner}")
    
    #skeletonize the mesh
    start_time_inner = time.time()
    print("starting creating skeleton")
    calc_return_value = cm.calcification(output_mesh[:-4])
    print("calc_return_value = " + str(calc_return_value))
    print("DONE creating skeleton")
    print(f"Skeleton time = {time.time() - start_time_inner}")
    
    return calc_return_value

    

In [7]:

segment_id = 648518346341366885
label_key = dict(segment_id=segment_id)
start_time = time.time()

skeletonize_Neuron_Meshlabserver(label_key)

print(f"Total time for Meshlabserver = {time.time() - start_time}")


Import time = 2.5806891918182373
starting voxelization
DONE voxelization
Voxelization time = 3.44388484954834
Writing Off file VOXEL time = 1.4593627452850342
starting meshlabserver Poisson surface reconstruction
subprocess_result= CompletedProcess(args='xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346341366885.off -o /notebooks/Pymeshfix_vs_Voxelization/temp/neuron_648518346341366885_mls.off -s /notebooks/Pymeshfix_vs_Voxelization/temp/remeshing_script_vox.mlx', returncode=0)
DONE meshlabserver Poisson surface reconstruction
Meshlabserver time = 11.937600135803223
starting creating skeleton
calc_return_value = 0
DONE creating skeleton
Skeleton time = 3.5491020679473877
Total time for Meshlabserver = 22.974225997924805


In [ ]:
"""
Advantages:
1) Speed
2) Voxelization may take care of wierd input mesh

Disadvantages:
1) Possible disconnected points (coudl be elimintated with size of voxels)
-- use c method for voxel (trimesh) --> voxel_mesh



"""